In [9]:
# Imports
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets
from torchvision import transforms
import tqdm
import numpy as np
from sklearn.metrics import f1_score

In [10]:
# Download the data

transform_train = transforms.Compose([
    # transforms.Resize(224),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    # transforms.Resize(224),
    transforms.ToTensor(),
])

training_data = datasets.CIFAR100(
    root = 'data',
    train = True,
    download = True,
    transform = transform_train
)

testing_data = datasets.CIFAR100(
    root = 'data',
    train = False,
    download = True,
    transform = transform_test
)
classes = training_data.classes

# DataLoader pytorch.org/docs/stable/data.html
batch_size = 32
training_dataloader = DataLoader(training_data, batch_size=batch_size)
testing_dataloader = DataLoader(testing_data, batch_size=batch_size)

# For CIFAR-100 we expect 60k total images
# 100 classes with 600 images each
# that's 50k training and 10k testing (500 training/100 testing per class)
print(f'training: {batch_size*len(training_dataloader)}, testing: {batch_size*len(testing_dataloader)}')

Files already downloaded and verified
Files already downloaded and verified
training: 50016, testing: 10016


In [11]:
# Define Training/Testing

# train
def train(dataloader, model, loss_fn, optimizer, device):
  model.train()
  # step being a training step
  for step, (X, y) in enumerate(dataloader):
    # send to CPU/GPU
    X = X.to(device)
    y = y.to(device)
    # model's prediction
    pred = model(X)
    # get loss
    loss = loss_fn(pred, y)
    # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # progress check
    if step % 100 == 0:
      print(f'step: {step}, loss: {loss.item()}')

# validation
def test(dataloader, model, loss_fn, device):
  num_steps = len(dataloader)
  model.eval()
  test_loss = 0
  correct = 0
  y_true = []
  y_predicted = []

  with torch.no_grad():
    for X, y in dataloader:
      X = X.to(device)
      y = y.to(device)
      pred = model(X)
      loss = loss_fn(pred, y)
      test_loss += loss.item()
      y_true.extend(y.cpu().numpy())
      y_hat = pred.argmax(1)
      y_predicted.extend(y_hat.cpu().numpy())
      correct_step = (y_hat == y).type(torch.float).sum().item()
      correct += correct_step

  test_loss /= num_steps # average loss
  correct = correct / (num_steps * batch_size)
  f1 = f1_score(y_true, y_predicted, average='macro')

  print()
  print(f'F1 Score: {f1}')
  print(f'Test Accuracy: {correct}')

In [ ]:
# get device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# create model
# https://pytorch.org/vision/main/models/generated/torchvision.models.densenet161.html
model = torchvision.models.densenet161(pretrained=True).to(device)
epochs = 20
lr = 5e-3
# optimize
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

print(f'Using {device}')
print(f'Optimizer: {optimizer}')
print(f'Epochs {epochs}')
print(f'Batch Size: {batch_size}')
print(f'Model Info: DenseNet161, pretrained=True')

for t in tqdm.tqdm(range(epochs)):
  print(f'Epoch\n\n{t}')
  train(training_dataloader, model, loss_fn, optimizer, device)
  test(testing_dataloader, model, loss_fn, device)
print('Done!')

Using cuda
Optimizer: SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.005
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)
Epochs 20
Batch Size: 32
Model Info: DenseNet161, pretrained=True


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch

0
step: 0, loss: 10.569035530090332
step: 100, loss: 6.157524585723877
step: 200, loss: 4.524527549743652
step: 300, loss: 3.9918622970581055
step: 400, loss: 3.9092514514923096
step: 500, loss: 3.6556596755981445
step: 600, loss: 3.5589535236358643
step: 700, loss: 3.1670079231262207
step: 800, loss: 3.085367441177368
step: 900, loss: 2.7142293453216553
step: 1000, loss: 3.3854563236236572
step: 1100, loss: 2.7954368591308594
step: 1200, loss: 2.8231310844421387
step: 1300, loss: 2.2252354621887207
step: 1400, loss: 2.3094258308410645
step: 1500, loss: 2.0886924266815186


  5%|▌         | 1/20 [02:40<50:47, 160.39s/it]


F1 Score: 0.4175311028826729
Test Accuracy: 0.42881389776357826
Epoch

1
step: 0, loss: 2.351440668106079
step: 100, loss: 2.2499895095825195
step: 200, loss: 2.6001229286193848
step: 300, loss: 2.334850549697876
step: 400, loss: 1.6023976802825928
step: 500, loss: 2.039435863494873
step: 600, loss: 1.8762636184692383
step: 700, loss: 1.8295605182647705
step: 800, loss: 1.5898487567901611
step: 900, loss: 1.7446368932724
step: 1000, loss: 1.9510353803634644
step: 1100, loss: 2.355841636657715
step: 1200, loss: 1.464699625968933
step: 1300, loss: 1.4554919004440308
step: 1400, loss: 1.4406678676605225
step: 1500, loss: 1.4896397590637207


 10%|█         | 2/20 [05:25<48:54, 163.05s/it]


F1 Score: 0.503381254780416
Test Accuracy: 0.5143769968051118
Epoch

2
step: 0, loss: 1.537580966949463
step: 100, loss: 1.4868738651275635
step: 200, loss: 1.7044000625610352
step: 300, loss: 1.6389195919036865
step: 400, loss: 1.066754937171936
step: 500, loss: 1.5143752098083496
step: 600, loss: 1.146845817565918
step: 700, loss: 1.2082693576812744
step: 800, loss: 1.1574392318725586
step: 900, loss: 1.377935767173767
step: 1000, loss: 1.370314598083496
step: 1100, loss: 1.739896297454834
step: 1200, loss: 1.1123261451721191
step: 1300, loss: 0.7336320877075195
step: 1400, loss: 0.7765432000160217
step: 1500, loss: 0.8085036873817444


 15%|█▌        | 3/20 [08:10<46:29, 164.08s/it]


F1 Score: 0.5468895652485322
Test Accuracy: 0.5493210862619808
Epoch

3
step: 0, loss: 0.916140079498291
step: 100, loss: 1.0493768453598022
step: 200, loss: 1.69439697265625
step: 300, loss: 1.380802869796753
step: 400, loss: 1.1994476318359375
step: 500, loss: 0.8548982739448547
step: 600, loss: 0.778311550617218
